In [1]:
addprocs(int(CPU_CORES / 2))

using DiscreteValueIteration
using SCAs

In [2]:
using SCAIterators, SCAConst

actions = ActionIterator(Actions)
for action in actions
    println(typeof(action), ": ", action)
end # for action

states = StateIterator(Xs, Ys, Bearings, Speeds, Speeds)
stateCount = 0
for state in states
    println(typeof(state), ": ", state)
    stateCount += 1
end # for state
println("nStates: ", NStates)
println("stateCount: ", stateCount)

Symbol: right20
Symbol: right10
Symbol: straight
Symbol: left10
Symbol: left20
Symbol: clearOfConflict
State: State(-2000.0,-2000.0,0.0,10.0,10.0,false)
State: State(-2000.0,-2000.0,0.0,10.0,15.0,false)
State: State(-2000.0,-2000.0,0.0,10.0,20.0,false)
State: State(-2000.0,-2000.0,0.0,15.0,10.0,false)
State: State(-2000.0,-2000.0,0.0,15.0,15.0,false)
State: State(-2000.0,-2000.0,0.0,15.0,20.0,false)
State: State(-2000.0,-2000.0,0.0,20.0,10.0,false)
State: State(-2000.0,-2000.0,0.0,20.0,15.0,false)
State: State(-2000.0,-2000.0,0.0,20.0,20.0,false)
State: State(-2000.0,-2000.0,1.5707963267948966,10.0,10.0,false)
State: State(-2000.0,-2000.0,1.5707963267948966,10.0,15.0,false)
State: State(-2000.0,-2000.0,1.5707963267948966,10.0,20.0,false)
State: State(-2000.0,-2000.0,1.5707963267948966,15.0,10.0,false)
State: State(-2000.0,-2000.0,1.5707963267948966,15.0,15.0,false)
State: State(-2000.0,-2000.0,1.5707963267948966,15.0,20.0,false)
State: State(-2000.0,-2000.0,1.5707963267948966,20.0,10.0